# ⚙️ Initiez-vous au MLOps (partie 1/2)

## 🫁 Detection du cancer du poumon

L'efficacité du système de prédiction du cancer aide les personnes à connaître leur risque de cancer à faible coût et les aide également à prendre la décision appropriée en fonction de leur statut de risque de cancer. Les données sont collectées à partir du site web du système de prédiction en ligne du cancer du poumon.

###  📦 Initialisation du notebook

In [31]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

import numpy as np
import pandas as pd
from src.visualization.visu_text import quick_df_info

### 🔗Préparation du dataset, merge des 2 fichiers csv

In [32]:
raw_df_patient = pd.read_csv("../data/raw/Patient_Lung_Cancer_Dataset.csv", sep=";")
quick_df_info(raw_df_patient, "Données brutes du patient")
raw_df_risks = pd.read_csv("../data/raw/Risks_Factors_Lung_Cancer_Dataset.csv", sep=";")
quick_df_info(raw_df_risks, "Données brutes des risques")


┌------------------------* Données brutes du patient *------------------------┐
├─------- Shape: (20000, 4) - Colonnes:
├─ID                        int64     
├─GENDER                    object    
├─AGE                       float64   
├─LUNG_CANCER               object    
└------------------------------------------------------------------------------┘

┌------------------------* Données brutes des risques *------------------------┐
├─------- Shape: (20000, 14) - Colonnes:
├─ID                        int64     
├─SMOKING                   object    
├─YELLOW_FINGERS            object    
├─ANXIETY                   object    
├─PEER_PRESSURE             object    
├─CHRONIC DISEASE           object    
├─FATIGUE                   object    
├─ALLERGY                   object    
├─WHEEZING                  object    
├─ALCOHOL CONSUMING         object    
├─COUGHING                  object    
├─SHORTNESS OF BREATH       object    
├─SWALLOWING DIFFICULTY     object    
├─CHEST PAIN

On effectue la jointure sur l'ID, pour avoir un fichier unique

In [33]:
df_raw_result = pd.merge(raw_df_patient, raw_df_risks, on='ID')
quick_df_info(df_raw_result, "Données brutes fusionnées")


┌------------------------* Données brutes fusionnées *------------------------┐
├─------- Shape: (20000, 17) - Colonnes:
├─ID                        int64     
├─GENDER                    object    
├─AGE                       float64   
├─LUNG_CANCER               object    
├─SMOKING                   object    
├─YELLOW_FINGERS            object    
├─ANXIETY                   object    
├─PEER_PRESSURE             object    
├─CHRONIC DISEASE           object    
├─FATIGUE                   object    
├─ALLERGY                   object    
├─WHEEZING                  object    
├─ALCOHOL CONSUMING         object    
├─COUGHING                  object    
├─SHORTNESS OF BREATH       object    
├─SWALLOWING DIFFICULTY     object    
├─CHEST PAIN                object    
└------------------------------------------------------------------------------┘


In [34]:
total_missing = df_raw_result.isnull().sum().sum()
if total_missing == 0:
    print("✅ Aucune valeur manquante détectée !")
else:
    print(f"⚠️  {total_missing} valeur(s) manquante(s) détectée(s)")

df_raw_result.info()

⚠️  2131 valeur(s) manquante(s) détectée(s)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     20000 non-null  int64  
 1   GENDER                 20000 non-null  object 
 2   AGE                    19942 non-null  float64
 3   LUNG_CANCER            20000 non-null  object 
 4   SMOKING                19607 non-null  object 
 5   YELLOW_FINGERS         20000 non-null  object 
 6   ANXIETY                20000 non-null  object 
 7   PEER_PRESSURE          20000 non-null  object 
 8   CHRONIC DISEASE        20000 non-null  object 
 9   FATIGUE                20000 non-null  object 
 10  ALLERGY                20000 non-null  object 
 11  WHEEZING               19057 non-null  object 
 12  ALCOHOL CONSUMING      19348 non-null  object 
 13  COUGHING               20000 non-null  object 
 14  SHORTNESS 

In [35]:
# Suppression des données manquantes
df_raw_result.dropna(inplace=True) # Pas de données manquantes dans ce dataset
df_raw_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17940 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     17940 non-null  int64  
 1   GENDER                 17940 non-null  object 
 2   AGE                    17940 non-null  float64
 3   LUNG_CANCER            17940 non-null  object 
 4   SMOKING                17940 non-null  object 
 5   YELLOW_FINGERS         17940 non-null  object 
 6   ANXIETY                17940 non-null  object 
 7   PEER_PRESSURE          17940 non-null  object 
 8   CHRONIC DISEASE        17940 non-null  object 
 9   FATIGUE                17940 non-null  object 
 10  ALLERGY                17940 non-null  object 
 11  WHEEZING               17940 non-null  object 
 12  ALCOHOL CONSUMING      17940 non-null  object 
 13  COUGHING               17940 non-null  object 
 14  SHORTNESS OF BREATH    17940 non-null  object 
 15  SWALLOW

### 🛠 Transformation des données en binaire 
- GENDER F=0, M=1
- LUNG_CANCER NO=0, YES=1 
- AUTRES NO=0, YES=1 

In [ ]:
cols = df_raw_result.columns.tolist()
delete = ["LUNG_CANCER", "GENDER", "AGE","ID"]
cols = [x for x in cols if x not in delete]
for col in cols:
    df_raw_result[col] = df_raw_result[col].map({'NO': 0, 'YES': 1}).astype(int)
df_raw_result["GENDER"] = df_raw_result["GENDER"].map({"F": 0, "M": 1})
df_raw_result["LUNG_CANCER"] = df_raw_result["LUNG_CANCER"].map({"NO": 0, "YES": 1})

df_raw_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17940 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     17940 non-null  int64  
 1   GENDER                 17940 non-null  int64  
 2   AGE                    17940 non-null  float64
 3   LUNG_CANCER            17940 non-null  int64  
 4   SMOKING                17940 non-null  int64  
 5   YELLOW_FINGERS         17940 non-null  int64  
 6   ANXIETY                17940 non-null  int64  
 7   PEER_PRESSURE          17940 non-null  int64  
 8   CHRONIC DISEASE        17940 non-null  int64  
 9   FATIGUE                17940 non-null  int64  
 10  ALLERGY                17940 non-null  int64  
 11  WHEEZING               17940 non-null  int64  
 12  ALCOHOL CONSUMING      17940 non-null  int64  
 13  COUGHING               17940 non-null  int64  
 14  SHORTNESS OF BREATH    17940 non-null  int64  
 15  SWALLOW

### 🧽 Sauvegarde du JDD nettoyé

In [38]:
df_raw_result.to_csv("../data/processed/survey_lung_cancer_clean.csv", index=False)